In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nbeats_keras.model import NBeatsNet as NBeatsKeras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from nbeats_pytorch.model import NBeatsNet as NBeatsPytorch
from keras.optimizers import RMSprop, Adam
import time
from keras.models import load_model
#from target_data_electronic70_7 import target_X, target_y ,test_X, test_y
#from m4databasis21_7 import base_domain,zt_in,zt_out,M4Meta,inputsize,train_12,train_12_y
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from tensorflow.keras.losses import Loss
import tensorflow as tf
#from m4databasis35_7_70_7 import train_35,train_35_y,train_70,train_70_y
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Add, Concatenate,Flatten,Reshape
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Lambda
from tensorflow.keras.callbacks import EarlyStopping

2024-10-18 14:05:00.274249: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-18 14:05:00.349252: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-18 14:05:00.349268: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-10-18 14:05:00.714586: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:

data = 'solor'
target_X= pd.read_csv(f"../data/{data}_train_input_7.csv").iloc[:,1:].values.astype(np.float32)
target_y =pd.read_csv(f"../data/{data}_train_output_7.csv").iloc[:,1:].values.astype(np.float32)

X_train = target_X[:-round(target_X.shape[0]*0.2),:].astype(np.float32)
y_train = target_y[:-round(target_y.shape[0]*0.2)].astype(np.float32)

X_train_val= target_X[-round(target_X.shape[0]*0.2):,:].astype(np.float32)
y_train_val =target_y[-round(target_y.shape[0]*0.2):].astype(np.float32)


test_X= pd.read_csv(f"../data/{data}_val_input_7.csv").iloc[:,1:].values.astype(np.float32)
test_y =pd.read_csv(f"../data/{data}_val_output_7.csv").iloc[:,1:].values.astype(np.float32)

#y_train.astype(np.float32)
backcast_length = X_train.shape[1]
forecast_length = y_train.shape[1]

In [3]:
#################################################################################
# loss SMAPE
class SMAPE(Loss):
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 예측 값의 차원을 맞춤
       # y_pred=tf.clip_by_value(y_pred, 1e-10, tf.reduce_max(y_pred))
       # y_true = tf.clip_by_value(y_true, 1e-10, tf.reduce_max(y_true))
        
        numerator = 100 * tf.abs(y_true- y_pred )
        denominator =  (tf.abs(y_true ) + tf.abs(y_pred))/2
        smape =  numerator /  denominator #tf.clip_by_value(denominator, 1e-10, tf.reduce_max(denominator))
        return tf.reduce_mean(smape)

#################################################################################
# loss MASE
class MASE(Loss):
    def __init__(self, training_data, period, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = self.calculate_scale(training_data, period)
    def seasonal_diff(data, period):
        return data[period:] - data[:-period]

    def calculate_scale(self, training_data, period):
        # 주기 차분 계산
        diff = seasonal_diff(training_data, period)
        scale = np.mean(np.abs(diff))
        return scale
    
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 차원 맞추기
        error = tf.abs(y_true - y_pred)
        return tf.reduce_mean(error / self.scale)

def seasonal_diff(data, period):
    return data[period:] - data[:-period]
#################################################################################
# 하이퍼파라미터 인자 설정
def hyperparameter():
    # 1 backcast
    # 2 forecast
    # 3 inputdim
    # 4 outputdim
    # 5 unit
    # 6 bacth size
    return X_train.shape[1],1,y_train.shape[1]

#################################################################################
# nbeats 모델 생성 함수
def build_model(input_timesteps,features,output_timesteps):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(input_timesteps, features)))
    model.add(LSTM(64, return_sequences=True))
    #model.add(LSTM(unit, return_sequences=True))
    # Use Lambda layer to select the last 'output_timesteps' outputs
    model.add(Lambda(lambda x: x[:, -24:, :]))
    model.add(TimeDistributed(Dense(1)))
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer)
    return model

#################################################################################
# 부트스트랩 샘플링
# 배깅

def train_bagging_models(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    input_timesteps,features,output_timesteps= hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = build_model(input_timesteps,features,output_timesteps)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 1, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=1, 
                  callbacks=[early_stop],
                validation_data = [X_train_val,y_train_val])
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys

#################################################################################




#################################################################################

def bagging_predict(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return np.median(predictions, axis=0)

def bagging_predict2(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return predictions

In [4]:
model_num = 10


mase_models = train_bagging_models(model_num, MASE(y_train,24),2000,10,8,0.001)
mape_models = train_bagging_models(model_num,'mape',2000,10,8,0.001)
smape_models = train_bagging_models(model_num, SMAPE(),2000,10,8,0.001)
mae_models = train_bagging_models(model_num, 'mae',2000,10,8,0.001)
mse_models = train_bagging_models(model_num, 'mse',2000,10,8,0.001)

2024-10-18 14:05:04.146979: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-10-18 14:05:04.147013: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ymlee2-desktop): /proc/driver/nvidia/version does not exist
2024-10-18 14:05:04.147484: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/2000
73/73 [==============================] - 8s 85ms/step - loss: 1.0891 - val_loss: 0.7985
Epoch 2/2000
73/73 [==============================] - 6s 80ms/step - loss: 0.8713 - val_loss: 0.6445
Epoch 3/2000
73/73 [==============================] - 6s 79ms/step - loss: 0.8008 - val_loss: 0.6320
Epoch 4/2000
73/73 [==============================] - 6s 77ms/step - loss: 0.7955 - val_loss: 0.5987
Epoch 5/2000
73/73 [==============================] - 6s 79ms/step - loss: 0.7860 - val_loss: 0.6276
Epoch 6/2000
73/73 [==============================] - 6s 77ms/step - loss: 0.7828 - val_loss: 0.5968
Epoch 7/2000
73/73 [==============================] - 6s 80ms/step - loss: 0.7610 - val_loss: 0.5878
Epoch 8/2000
73/73 [==============================] - 6s 82ms/step - loss: 0.7590 - val_loss: 0.6205
Epoch 9/2000
73/73 [==============================] - 6s 80ms/step - loss: 0.7655 - val_loss: 0.5867
Epoch 10/2000
73/73 [==============================] - 6s 78ms/step - loss: 0.7606 - val_lo

In [5]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
pred4,_=mae_models
pred5,_=mse_models

smape_predictions = bagging_predict2(pred1, X_train_val)
mase_predictions =  bagging_predict2(pred2, X_train_val)
mape_predictions =  bagging_predict2(pred3, X_train_val)
mae_predictions = bagging_predict2(pred4, X_train_val)
mse_predictions =  bagging_predict2(pred5, X_train_val)


concat_G = np.concatenate([smape_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
sMAPE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mape_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MAPE= np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mase_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MASE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mae_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MAE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mse_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MSE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)


MSE, MASE, MAE, MAPE, sMAPE

5/5 [==============================] - 0s 29ms/step


(0.40127, 0.4218, 0.42332, 0.96668, 0.42786)

In [6]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
pred4,_=mae_models
pred5,_=mse_models

smape_predictions2 = bagging_predict2(pred1, test_X)
smape_predictions2 = np.median(np.concatenate([smape_predictions2],axis=0),axis=0)

mase_predictions2 =bagging_predict2(pred2, test_X)
mase_predictions2 = np.median(np.concatenate([mase_predictions2],axis=0),axis=0)

mape_predictions2 =bagging_predict2(pred3, test_X)
mape_predictions2 = np.median(np.concatenate([mape_predictions2],axis=0),axis=0)

mae_predictions2 = bagging_predict2(pred4,test_X)
mae_predictions2 = np.median(np.concatenate([mae_predictions2],axis=0),axis=0)

mse_predictions2 =bagging_predict2(pred5,test_X)
mse_predictions2 = np.median(np.concatenate([mse_predictions2],axis=0),axis=0)


#concat_mase = np.concatenate([np.nan_to_num(np.array(mase_predictions2), nan=0)])
#fin_pred_mase = np.median(concat_mase,axis=1)

#concat_mape = np.concatenate([np.nan_to_num(np.array(mape_predictions2), nan=0)])
#fin_pred_mape = np.median(concat_mape,axis=1)

#concat_smape = np.concatenate([np.nan_to_num(np.array(smape_predictions2), nan=0)])
#fin_pred_smape = np.median(concat_smape,axis=1)

#concat_mae = np.concatenate([np.nan_to_num(np.array(mae_predictions2), nan=0)])
#fin_pred_mae = np.median(concat_mae,axis=1)

#concat_mse = np.concatenate([np.nan_to_num(np.array(mse_predictions2), nan=0)])
#fin_pred_mse = np.median(concat_mse,axis=1)

performance = np.array([MAE, MAPE,sMAPE,MSE,MASE])
beta = 3 # 조정 파라미터
weights = np.exp(-beta * performance)

gd= np.concatenate([mae_predictions2,
                    mape_predictions2,
                   smape_predictions2,
                   mse_predictions2,
                   mase_predictions2],axis=0)
#gd=np.median(gd,axis=2)
normalized_weights = weights / np.sum(weights)

# 각 모델의 예측값에 가중치를 부여하여 앙상블 예측 생성
ensemble_prediction = np.dot(normalized_weights, gd.reshape(5,-1))
print('############################################################################################') 
print('############################################################################################') 
pd.DataFrame(ensemble_prediction.flatten()).to_csv('exp7/LSTM.csv')


print('exp',np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten())).round(5))

12/12 [==============================] - 0s 32ms/step
############################################################################################
############################################################################################
exp 0.47407


In [7]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
pred4,_=mae_models
pred5,_=mse_models

smape_predictions_G = bagging_predict2(pred1, test_X)
mase_predictions_G = bagging_predict2(pred2,test_X)
mape_predictions_G = bagging_predict2(pred3,test_X)
mae_predictions_G = bagging_predict2(pred4, test_X)
mse_predictions_G = bagging_predict2(pred5,test_X)


print('############################################################################################') 
print('############################################################################################') 

concat_G = np.concatenate([smape_predictions_G, mase_predictions_G,mape_predictions_G,mae_predictions_G,mse_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('all',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5)) 

concat_G = np.concatenate([smape_predictions_G, mase_predictions_G,mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('original',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mse_predictions_G, mase_predictions_G,mae_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('best',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mse_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mse',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))


concat_G = np.concatenate([mase_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mase',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mae_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mae',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mape',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([smape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('smape',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

12/12 [==============================] - 0s 32ms/step
############################################################################################
############################################################################################
all 0.47959
original 0.48744
best 0.47817
mse 0.46746
mase 0.48523
mae 0.48684
mape 1.08731
smape 0.49103


In [8]:
eee = []
for i in range(1,20):
    weights = np.exp(-i* performance)
    normalized_weights = weights / np.sum(weights)
    ensemble_prediction = np.dot(normalized_weights, gd.reshape(5,-1))
    eee.append((i,np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten().round(5)))))
    #print(f'exp_beta{i}',np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten())).round(5))

eee

[(1, 0.4793389),
 (2, 0.47443143),
 (3, 0.47406745),
 (4, 0.47484627),
 (5, 0.47562924),
 (6, 0.47616154),
 (7, 0.4764584),
 (8, 0.4765883),
 (9, 0.47661152),
 (10, 0.47656977),
 (11, 0.4764896),
 (12, 0.476387),
 (13, 0.47627136),
 (14, 0.47614828),
 (15, 0.47602093),
 (16, 0.47589105),
 (17, 0.47575995),
 (18, 0.47562814),
 (19, 0.47549605)]